In [1]:
############### Acknowledgement ###############
# set your browser download location to your desire location because all files will be downloaded to the download location setted in your browser.

# Thanks -> pythonjar, MariyaSha and some other stackoverflow members...

In [2]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [9]:
# disable all alerts from facebook while scrapping
options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
options.add_experimental_option("prefs",prefs)

#############################################################################
# specify the path to chromedriver.exe (download and save on your computer) #
#############################################################################

############################## WINDOWS ######################################
# service = Service('C:/Users/shahr/chromedriver.exe')                      #
# driver = webdriver.Chrome(service=service, options=options)               #
#############################################################################

############################## LINUX ########################################
# driver = webdriver.Chrome('chromedriver', options=options) # linux        #
#############################################################################
service = Service('C:/Users/shahr/chromedriver.exe')
driver = webdriver.Chrome(service=service, options=options)

# for obvious reason 😁
with open('.secret', encoding='utf-8') as f:
    secrets = [line.strip() for line in f.readlines()] # 0 -> username, 1 -> password, 2 -> download directory (for checking)

In [10]:
# open the webpage
driver.get("http://www.facebook.com")

# target credentials
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='email']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='pass']")))

# enter username and password
username.clear()
username.send_keys(secrets[0])
password.clear()
password.send_keys(secrets[1])

# target thesrc login button and click it
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

###############################################################################################################################
# It should be logged in! -> if you have 2fa then you have to authorize it manually and then run the remaining cells manually #
###############################################################################################################################

In [11]:
log_file_location = 'bot.log'

#########################################################
# sleep is important because if we scrape too fast then #
# facebook will detect the bot and block this account   #
#########################################################
def waitNSeconds(sleep_time = 1):
    time.sleep(sleep_time)

def writeToFile(text):
    with open(log_file_location, 'a', encoding='utf-8') as f:
        f.write(text + "\n")

###################################################################
# scroll down to load more files                                  #
# wait 60s before determining that there is no more files to load #
###################################################################
def loadMoreFiles(files_to_load, identifier, timeout = 60, n_scroll = 1):
    for _ in range(n_scroll): # scroll for n times
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        seconds = 0
        current_len = len(files_to_load)

        while (current_len >= len(files_to_load)) and seconds < timeout:
            time.sleep(1.5) # wait every 1.5s before checking again -> because 1s is too fast :V

            files_to_load.extend([
                element for element in driver.find_elements(
                    By.XPATH, identifier
                ) if element not in files_to_load
            ])

            seconds += 1.5
        
    return files_to_load

def waitToFinishDownload(directory, timeout, nfiles=None):
    """
    Wait for downloads to finish with a specified timeout.

    Args
    ----
    directory : str
        The path to the folder where the files will be downloaded.
    timeout : int
        How many seconds until it stops waiting.
    nfiles : int, defaults to None
        If provided, also wait for the expected number of files.

    """
    
    print("\nWaiting for {} files to be downloaded\n".format(nfiles))

    seconds = 0
    dl_wait = True

    while dl_wait and seconds < timeout:
        time.sleep(0.5) # check every 0.5s
        dl_wait = False
        files = os.listdir(directory)
        
        # if nfiles and len(files) != nfiles:
        # if nfiles and len(files) < nfiles:
        #     dl_wait = True

        for fname in files:
            if fname.endswith('.crdownload'): # as partial downloaded files will be of ".crdownload" extension for chromium based browsers
                dl_wait = True
                break

        seconds += 0.5
    
    if seconds >= timeout:
        seconds = -1
        print("Your connection is too slow or you are not connected! Try again later. Closing the connection...")
    else:
        print("Continuing after {}s...\n".format(seconds))

    return seconds

# Resume from last downloaded file
def checkDownloadHistory(directory, files_in_directory):
    with open(directory, 'r', encoding='utf-8') as f:
        total_lines = files_in_directory
        file_entries = f.readlines()

        while total_lines:
            total_lines -= 1

            try:
                last_entry = file_entries[total_lines]
                last_entry = str(last_entry).split(' --- ')

                # [1:-1] is to strip out the single quote & strip() for extra spaces
                last_file_name = last_entry[0].strip()[1:-1]
                last_file_date = last_entry[2].strip()[1:-1]

                print("Download history: Found. Download will resume from '{}' file".format(last_file_name))
                
                return last_file_name, last_file_date
            except:
                last_file_name = False

    if not last_file_name: print("Download history: Not Found")

    return False, False

In [12]:
# waitNSeconds(5) # enable this if this script is fully automated for your case
driver.get("https://www.facebook.com/groups/201623576939858/files/")

download_thread_count = 3 # this value will ensure how many downloads will happen concurrently

# to wait until the element is ready -> Explicit Waits
wait = WebDriverWait(driver, 10)

##################################################################################
# targets identifiers                                                            #
# this identifiers will change continuously so update it according to your needs #
##################################################################################
fileOption_xpath = "//div[@aria-label='File options']" # find 15 per-scroll
fileName_xpath = "//span[@class='d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v lrazzd5p oo9gr5id hzawbc8m']" # find 15 per-scroll
fileTypeDate_xpath = "//span[@class='d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d9wwppkn iv3no6db e9vueds3 j5wam9gi b1v8xokw oo9gr5id hzawbc8m']" # find 30 per-scroll
# sortButtons_xpath = "//div[@class='l9j0dhe7 du4w35lb j83agx80 pfnyh3mw taijpn5t bp9cbjyn owycx6da btwxx1t3 kt9q3ron ak7q8e6j isp2s0ed ri5dt5u2 rt8b4zig n8ej3o3l agehan2d sk4xxmp2 rq0escxv d1544ag0 tw6a2znq tdjehn4e tv7at329']" # find 3 per-scroll

#########################################################################
# as it doesn't sort by file type reliably 😑
# sortButtons = wait.until(
#     EC.presence_of_all_elements_located((By.XPATH, sortButtons_xpath))
# )

# # sort by file_type (PDF at the top in my case)
# driver.execute_script("arguments[0].click();", sortButtons[1])
#########################################################################

In [13]:
# initial target search
file_option_buttons = wait.until(
    EC.presence_of_all_elements_located((By.XPATH, fileOption_xpath))
)
file_names = wait.until(
    EC.presence_of_all_elements_located((By.XPATH, fileName_xpath))
)
file_types_and_dates = wait.until(
    EC.presence_of_all_elements_located((By.XPATH, fileTypeDate_xpath))
)

j, k = 0, 1
registered_files_count = sum(1 for line in open(log_file_location, encoding='utf-8')) # total files registered in the log file
resume_download = not registered_files_count # if there is log then download will not resume without checking

# Resume from last downloaded file
if registered_files_count:
    last_file_name, last_file_date = checkDownloadHistory(log_file_location, registered_files_count)

for idx, button in enumerate(file_option_buttons):
    try:
        _name = file_names[idx].text
        _type = file_types_and_dates[j].text
        _date = file_types_and_dates[k].text

        # as file_types_and_dates contains both type and date
        j += 2; k += 2

        # scroll after 15th button click to load more files (as 15 files load per scroll)
        if (idx + 1) % 15 == 0:
            file_option_buttons = loadMoreFiles(file_option_buttons, fileOption_xpath)

            file_names.extend([
                name for name in driver.find_elements(
                    By.XPATH, fileName_xpath
                ) if name not in file_names
            ])

            file_types_and_dates.extend([
                type_date for type_date in driver.find_elements(
                    By.XPATH, fileTypeDate_xpath
                ) if type_date not in file_types_and_dates
            ])

        ######################################################################################################################
        # download pdf files only                                                                                            #
        # cannot check it before the scroll because there is a possibility to have more than one page worth of non-pdf files #
        ######################################################################################################################
        if not _type == "PDF": continue

        #################################################################################################################
        # to skip already downloaded files                                                                              #
        # if first check is False then others will not be checked so you don't have to ->                               #
        # worry about having last_file_name, last_file_date being initialized because of resume_download check at first #
        #################################################################################################################
        if not resume_download and last_file_name and _name == last_file_name and _date == last_file_date:
            print("Resuming...\n")
            resume_download = True
            continue # for skipping the last entry too

        if not resume_download:
            print("Skipping... {}/{} ---> {}".format(idx+1, len(file_option_buttons), _name))
            continue
        
        ##########################################################################################################
        # waiting after every 3 donwloads                                                                        #
        # we have to wait for the files to be downloaded, if we continue without it there maybe some files       #
        # that couldn't be downloaded on time (before the link expire) and later on those files can't be resumed #
        ##########################################################################################################
        if (idx + 1) % download_thread_count == 0:
            download_time = waitToFinishDownload(secrets[2], 600, registered_files_count) # waiting for 600s before closing the connection for slow/no internet

            if download_time == -1: break # stoping the scrapper incase of network failure

        ###################################### download script starts ######################################

        driver.execute_script("arguments[0].click();", button)
        # wait.until(EC.element_to_be_clickable(button)).click() # this was giving me -> ElementClickInterceptedException

        waitNSeconds(0.8)

        try:
            download_link = wait.until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='https://www.facebook.com/download/']"))
            )
            driver.execute_script("arguments[0].target='_self';", download_link) # to prevent it from opening into a new tab
            driver.execute_script("arguments[0].click();", download_link)
            
            registered_files_count += 1 # keeping track of downloaded files

        except:
            writeToFile("***** Warning: No download link found at {} *****".format(idx+1))
            
        ###################################### download script finished ###################################

        # updating log
        log = '"{}" --- "{}" --- "{}"'.format(_name, _type, _date)
        writeToFile(log)

        print("{} --- {} - {}".format(_name, _date, len(file_option_buttons)))

        #################################################################
        # waiting at least 2s for safety sake 😅                        #
        # waiting (1.2+0.8+calculation_time) -> 2s+ after each download #
        #################################################################
        waitNSeconds(1.2)

    except Exception as e:
        log = "***** ERROR at {}, date: {}: {} *****".format(idx+1, _date, e)
        
        print(log)
        writeToFile(log)

# driver.close() # not closing because I had to run this cell multiple times during test
print("\nCompleted. Total scrapped file:", registered_files_count)

Download history: Found. Download will resume from 'আনন্দমেলা ১৮১১৮১.pdf' file
Skipping... 1/15 ---> জলছবি - সুচিত্রা ভট্টাচার্য.pdf
Skipping... 2/15 ---> বেগম আখতার.pdf
Skipping... 3/15 ---> রামানুজের জ্যোতির্বলয় – উৎপল মুখোপাধ্যায়.pdf
Skipping... 4/15 ---> আয়নামহল - সুচিত্রা ভট্টাচার্য.pdf
Skipping... 5/15 ---> অনেক বেবুন - অমিয়ভূষণ মজুমদার.pdf
Skipping... 6/15 ---> ৩০০ বছরের কলকাতা পটভূমি ও ইতিকথা - ডঃ অতুল সুর .pdf
Skipping... 7/15 ---> রম্যগণিত.pdf
Skipping... 8/15 ---> অঘটনা - অমিয়ভূষণ মজুমদার.pdf
Skipping... 9/15 ---> আনন্দমেলা২০০৩১১.pdf
Skipping... 10/15 ---> আনন্দমেলা ২০০২১৫.pdf
Skipping... 11/15 ---> আনন্দমেলা১২০১২০০০.pdf
Skipping... 12/15 ---> আনন্দমেলা ২০০৭১৫.pdf
Skipping... 13/15 ---> আনন্দমেলা২০১০১৫.pdf
Skipping... 14/15 ---> আনন্দমেলা ০২০৪০৮.pdf
Skipping... 15/30 ---> আনন্দমেলা২০০৯২০০০.pdf
Skipping... 16/30 ---> আনন্দমেলা১৪০৩০১.pdf
Skipping... 17/30 ---> আনন্দমেলা২০০৪০৩.pdf
Skipping... 18/30 ---> আনন্দমেলা ২১০৬২০০০.pdf
Skipping... 19/30 ---> উনবিংশ শতকের গীতিকবিতা সংক